This is the API documentation for Notes-api

In [18]:


val baseUrl = "http://127.0.0.1:8081"



In [51]:
//Ktor Client initialization

import io.ktor.client.HttpClient
import io.ktor.client.plugins.contentnegotiation.ContentNegotiation
import io.ktor.client.request.get
import io.ktor.serialization.kotlinx.json.json

val ktorClient = HttpClient() {
    install(ContentNegotiation) { json() }
}

In [34]:
import io.ktor.client.statement.HttpResponse
import kotlinx.coroutines.runBlocking

// Base URL
runBlocking {
    val lol = ktorClient.get(baseUrl)
    print(lol.deserializeJson())
}

Hello World

In [70]:
runBlocking {
    val notes = ktorClient.get("$baseUrl/notes")
    println("response: $notes")
    println("data: ${notes.deserializeJson()}")
}

response: HttpResponse[http://127.0.0.1:8081/notes, 200 OK]
data: [{"id":"433a1af4-bf84-485f-bb78-72e8b66873c1","title":"Legend","content":"hahahahah","createdAt":1753873160151,"updatedAt":1753873181963}]


In [ ]:
import io.ktor.client.request.post

runBlocking {
    val postNote = ktorClient.post("$baseUrl/notes")
    println("response: $postNote")
    println("data: ${postNote.deserializeJson()}")
}